In [16]:
#get_ipython().magic('reset -f')
###### project paths
project_name = "product_class_single_new"
project_path = "/001/usuarios/ajason08/notebooks/prod_class/"
temp_path = "/001/usuarios/ajason08/notebooks/temp_jason/"
#!ls "{project_path}"
#!ls "{temp_path}"

reloadables = {
  "ajnlp_functions" : "1ihLk7IagdfUl5gMV-odGiNyo9qqljNFy",
  "ajnlp_models" : "1aC_XblzZ6-dyskLnhNIe4HHc_IkbJZaG",
  "ajutil" : "176qb4Fh8MBUnlKDjCUFWd7zhpCHVG_mS",
  "Jpandas": "12jzHusu8hummWi4PgwhBsed1-Djs6u6i"
}

import gdown
def jgdown(drive_id, outpath="./", islist=False):
  if islist:
    for single_drive_id in drive_id:
      url = "https://drive.google.com/uc?id="+single_drive_id
      gdown.download(url, outpath, quiet=False)    
  else:
    url = "https://drive.google.com/uc?id="+drive_id
    gdown.download(url, outpath, quiet=False)
      
import sys, importlib
def reloader(to_reload, remote, drive_id=None, output="./"):
  try:
      if remote: jgdown(drive_id, output)
      importlib.reload(sys.modules[to_reload])
  except KeyError: 
      raise ImportError(f"Module {to_reload} not loaded. Can't reload"
                        f" or not reload-file found")  
    
#jgdown(list(reloadables.values()), outpath=temp_path, islist=True)
#jgdown(reloadables['ajutil'], outpath=temp_path)

toreaload = "ajnlp_models"; reloader(toreaload, remote=True, drive_id=reloadables[toreaload], output= temp_path) 

## download this notebook as executable script
#!jupyter nbconvert --to script product_class_light.ipynb

#from IPython import get_ipython


Downloading...
From: https://drive.google.com/uc?id=1aC_XblzZ6-dyskLnhNIe4HHc_IkbJZaG
To: /001/usuarios/ajason08/notebooks/temp_jason/ajnlp_models.py
100%|██████████| 6.96k/6.96k [00:00<00:00, 16.7MB/s]


In [2]:
# Jupyter packages
from IPython.display import display

# Deep learning packages
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data, datasets
from torchtext.vocab import Vectors as myCustomEmbeddings

# specific DL models
from transformers import BertTokenizer, BertModel, AutoTokenizer

# scikit packages
import numpy as np
import pandas as pd

# Auxiliar packages
import random, sys
from IPython.display import display, HTML, Markdown

# personal algorithms
sys.path.append(temp_path)
import ajnlp_functions as ajf# such as train, test, score, etc
import ajnlp_models as jmodels
import ajnlp_models # such as cnn, gru, etc
import ajutil # utilities such as stopwatch
import Jpandas # pandas convenient function

# CNN = ajnlp_models.CNN
# GRU = ajnlp_models.GRU
# BERTGRU = ajnlp_models.BERTGRU
# BERTAlone = ajnlp_models.BERTAlone

In [19]:
# logger settings
model_filename = project_name + '_model.pt'
saved_model_path = temp_path + model_filename
# Data loading
train_dev_file= project_path +  "train_dev.csv"
#train_file= project_path + "DiscriminAtt/training/train.txt"
#dev_file= project_path + "DiscriminAtt/training/validation.txt"
GOLD = True

# environment setting
device = torch.device('cuda') # cuda cpu ('cuda' if torch.cuda.is_available() else 'cpu'))
torch.cuda.set_device(3)
SEED = 1234

#predictions
METRIC_FOCUS = "fscore" # loss accuracy fscore #last
METRIC_AVERAGE = "weighted" # None micro macro weighted

# Contextual embedding settings
#CONTEXTUAL_EMBED= False # True  False(bert)
CONTEXTUAL_MODEL = 'bert-base-uncased'
TOKENIZER = AutoTokenizer.from_pretrained(CONTEXTUAL_MODEL)

# hyper-parameters
hopt = {
  "RUN": 1,
  "EPOCHS": 1,
  "VOCAB_SIZE" : 15_000, #15_000
  "BATCH_SIZE" : 64, # 64 128(using-bert)
  ### Model hyperparameters
  # regularization
  "DROPOUT" : 0.2, # 0.25(gru) 0.5(cnn)
  #CNN
  "N_FILTERS" : 100, #100
  "FILTER_SIZES" : [2,3],#[2,3,4]
  # RNN
  "HIDDEN_DIM" : 256, # 256 64(cheap-bert)
  "N_LAYERS" : 1,
  "BIDIRECTIONAL" : True,
  # BERT
  "BERT_FROZEN" : True,
  "BERT_POOLED" : False
  }
BENCHMARK = True

In [4]:
# loading embeddings
embedding_file1 = temp_path + "w2v_dim128_win1_minc1_epocs_10.w2v"
vec = myCustomEmbeddings(embedding_file1) # "glove.6B.100d"

In [ ]:
def create_bert_alone2(frozen=True):
  bert = BertModel.from_pretrained(CONTEXTUAL_MODEL)
  OUTPUT_DIM = len(LABEL.vocab)
  
  model = BERTAlone2(bert, OUTPUT_DIM, hopt["DROPOUT"])

  #freezing bert will reduce many millions of parameters to learn
  if frozen: 
    for name, param in model.named_parameters(): 
      if name.startswith('bert'): param.requires_grad = False

  # criterion and optimizer
  criterion = nn.CrossEntropyLoss()
  criterion = criterion.to(device)
  optimizer = optim.Adam(model.parameters())
  model = model.to(device)
  ajf.count_parameters(model)
  return model, optimizer, criterion

class BERTAlone2(nn.Module):
  def __init__(self, bert, output_dim, dropout, frozen=False):      
    super().__init__()      
    self.bert = bert      
    embedding_dim = bert.config.to_dict()['hidden_size']      
    self.dropout = nn.Dropout(dropout)
    self.out = nn.Linear(embedding_dim, output_dim)
      
  def forward(self, text):                 
    embedded = self.bert(text)[0]   
    embedded = torch.mean(embedded, 1) # aggregating all wordvectors per sentences
    embedded = self.dropout(embedded)  
    output = self.out(embedded)      
    return output
#     #freezing bert will reduce many millions of parameters to learn
#     print(frozen)
#     if frozen:  
#       print(frozen)
#       for name, param in self.named_parameters(): 
#           if name.startswith('bert'): param.requires_grad = False    

In [ ]:
# def create_cnn():
#   # Model: main arguments
#   INPUT_DIM = len(TEXT.vocab)
#   OUTPUT_DIM = len(LABEL.vocab)
#   EMBEDDING_DIM = len(TEXT.vocab.vectors[0])
#   PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

#   model = CNN(INPUT_DIM, OUTPUT_DIM, EMBEDDING_DIM, PAD_IDX,
#               hopt["N_FILTERS"], hopt["FILTER_SIZES"], hopt["DROPOUT"])
#   # initial weights
#   UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
#   model.embedding.weight.data.copy_(TEXT.vocab.vectors)
#   model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
#   model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
#   # criterion and optimizer
#   criterion = nn.CrossEntropyLoss()
#   criterion = criterion.to(device)
#   optimizer = optim.Adam(model.parameters())
#   model = model.to(device)
  
#   return model, optimizer, criterion

# def create_gru():
#   # Model: main arguments
#   INPUT_DIM = len(TEXT.vocab)
#   OUTPUT_DIM = len(LABEL.vocab)
#   EMBEDDING_DIM = len(TEXT.vocab.vectors[0])

#   model = GRU(INPUT_DIM,EMBEDDING_DIM, hopt["HIDDEN_DIM"], hopt["N_LAYERS"], hopt["BIDIRECTIONAL"],
#                            hopt["DROPOUT"], OUTPUT_DIM)

#   # initial weights
#   UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
#   PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
#   model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
#   model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
#   model.embedding.weight.data.copy_(TEXT.vocab.vectors)
  
#   # criterion and optimizer
#   criterion = nn.CrossEntropyLoss()
#   criterion = criterion.to(device)
#   optimizer = optim.Adam(model.parameters())
#   model = model.to(device)
#   return model, optimizer, criterion

# def create_bert_gru(frozen=False):
#   bert = BertModel.from_pretrained(CONTEXTUAL_MODEL)
#   OUTPUT_DIM = len(LABEL.vocab)
  
#   model = BERTGRU2(bert, hopt["HIDDEN_DIM"], hopt["N_LAYERS"], hopt["BIDIRECTIONAL"],
#                   hopt["DROPOUT"], OUTPUT_DIM)

#   #freezing bert will reduce many millions of parameters to learn
#   if frozen: 
#     for name, param in model.named_parameters(): 
#       if name.startswith('bert'): param.requires_grad = False

#   # criterion and optimizer
#   criterion = nn.CrossEntropyLoss()
#   criterion = criterion.to(device)
#   optimizer = optim.Adam(model.parameters())
#   model = model.to(device)
#   return model, optimizer, criterion

# def create_bert_alone(frozen=False):
#   bert = BertModel.from_pretrained(CONTEXTUAL_MODEL)
#   OUTPUT_DIM = len(LABEL.vocab)
  
#   model = BERTAlone2(bert, hopt["DROPOUT"], OUTPUT_DIM)

#   #freezing bert will reduce many millions of parameters to learn
#   if frozen: 
#     for name, param in model.named_parameters(): 
#       if name.startswith('bert'): param.requires_grad = False

#   # criterion and optimizer
#   criterion = nn.CrossEntropyLoss()
#   criterion = criterion.to(device)
#   optimizer = optim.Adam(model.parameters())
#   model = model.to(device)
#   return model, optimizer, criterion

In [ ]:
# def create_cnn2(input_dim, output_dim, embedding_dim, first_vec, pad_idx, unk_idx,
#                 nfilters, filter_sizes, dropout):

#   model = CNN(input_dim, output_dim,embedding_dim, pad_idx,
#                 nfilters, filter_sizes, dropout)
#   # initial weights  
#   model.embedding.weight.data.copy_(first_vec)
#   model.embedding.weight.data[unk_idx] = torch.zeros(embedding_dim)
#   model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)
#   return model

# def create_gru2():
#   # Model: main arguments
#   INPUT_DIM = len(TEXT.vocab)
#   OUTPUT_DIM = len(LABEL.vocab)
#   EMBEDDING_DIM = len(TEXT.vocab.vectors[0])

#   model = GRU(INPUT_DIM,EMBEDDING_DIM, hopt["HIDDEN_DIM"], hopt["N_LAYERS"], hopt["BIDIRECTIONAL"],
#                            hopt["DROPOUT"], OUTPUT_DIM)

#   # initial weights
#   UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
#   PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
#   model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
#   model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
#   model.embedding.weight.data.copy_(TEXT.vocab.vectors)
  
#   # criterion and optimizer


#   return model

# def create_bert_gru2(frozen=True):
  
#   OUTPUT_DIM = len(LABEL.vocab)
  
#   model = BERTGRU(bert, hopt["HIDDEN_DIM"], hopt["N_LAYERS"], hopt["BIDIRECTIONAL"],
#                   hopt["DROPOUT"], OUTPUT_DIM)

#   #freezing bert will reduce many millions of parameters to learn
#   if frozen: 
#     for name, param in model.named_parameters(): 
#       if name.startswith('bert'): param.requires_grad = False

#   # criterion and optimizer
#   criterion = nn.CrossEntropyLoss()
#   criterion = criterion.to(device)
#   optimizer = optim.Adam(model.parameters())
#   model = model.to(device)
#   return model, optimizer, criterion

# def create_bert_alone2(frozen=True):
#   bert = BertModel.from_pretrained(CONTEXTUAL_MODEL)
#   OUTPUT_DIM = len(LABEL.vocab)
  
#   model = BERTAlone2(bert, OUTPUT_DIM, hopt["DROPOUT"])

#   #freezing bert will reduce many millions of parameters to learn
#   if frozen: 
#     for name, param in model.named_parameters(): 
#       if name.startswith('bert'): param.requires_grad = False

#   # criterion and optimizer
#   criterion = nn.CrossEntropyLoss()
#   criterion = criterion.to(device)
#   optimizer = optim.Adam(model.parameters())
#   model = model.to(device)
#   return model, optimizer, criterion

In [5]:
train_df, dev_df = Jpandas.row_splitter(train_dev_file,
                                               "dataset", ["train","dev"],
                                                path_input=True, sep="\t")

# display(pd.read_csv(train_dev_file, "\t").head(4))
# display(train_df.head(4))
# display(dev_df.head(4))

target_name = "lvl3"
input_names = ["description_new"]
column_names= [target_name] + input_names
tr_val_dev_datasets, fields = ajf.data_wrap(column_names, ["label", "contextual"],
                                           target_name, train_df, dev_df, SEED,
                                           hopt["VOCAB_SIZE"], hopt["BATCH_SIZE"],
                                           vec, tokenizer=TOKENIZER)

LABEL = fields[target_name]
TEXT =  fields[input_names[0]] # first input_name

In [17]:
def model_creation(model_kind, verbosity=False):
  criterion = nn.CrossEntropyLoss()
  criterion = criterion.to(device)      
  OUTPUT_DIM = len(LABEL.vocab)  
  ajf.seed_torch(seed=SEED) # BENCHMARK    
  
  if "bert" in model_kind:
    bert = BertModel.from_pretrained(CONTEXTUAL_MODEL)
    
    if model_kind == "bert":      
      model = jmodels.BERT_only(bert, OUTPUT_DIM, hopt["DROPOUT"],
                                hopt["BERT_POOLED"], hopt["BERT_FROZEN"])
    elif model_kind == "bert_gru":
      model = jmodels.BERTGRU(bert, OUTPUT_DIM,
                          hopt["HIDDEN_DIM"], hopt["N_LAYERS"], hopt["BIDIRECTIONAL"],
                             hopt["DROPOUT"], hopt["BERT_FROZEN"])          
#       return create_bert_gru2()
    else: raise ValueError(f'model {model_kind} is not supported')          
  
  else:  
    INPUT_DIM = len(TEXT.vocab)  
    EMBEDDING_DIM = len(TEXT.vocab.vectors[0])
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    FIRST_VEC = TEXT.vocab.vectors    
    
    if model_kind == "cnn": 
      model = jmodels.CNN(INPUT_DIM, OUTPUT_DIM, EMBEDDING_DIM,                        
                          hopt["N_FILTERS"], hopt["FILTER_SIZES"], hopt["DROPOUT"],
                           FIRST_VEC, PAD_IDX, UNK_IDX)
    elif model_kind == "gru":
      model = jmodels.GRU(INPUT_DIM, OUTPUT_DIM, EMBEDDING_DIM,
                          hopt["HIDDEN_DIM"], hopt["N_LAYERS"], hopt["BIDIRECTIONAL"],
                             hopt["DROPOUT"],
                           FIRST_VEC, PAD_IDX, UNK_IDX)        
    else: raise ValueError(f'model {model_kind} is not supported')          
  if verbosity: ajf.count_parameters(model)
  
  optimizer = optim.Adam(model.parameters())
  model = model.to(device)  
  return model, optimizer, criterion

def deploy_experiment(model, optimizer, criterion):

  train_iter, valid_iter, dev_iter = data.BucketIterator.splits(
    tr_val_dev_datasets,
    sort = False,
    batch_size = hopt["BATCH_SIZE"],
    device = device, 
    #shuffle=False
  )
  
  train_report = ajf.train_model(model, optimizer, criterion,
                                 train_iter, valid_iter,
                                 input_names, target_name,
                                 saved_model_path, 
                                 selector= METRIC_FOCUS, avg = METRIC_AVERAGE,
                                 epochs=hopt["EPOCHS"], verbosity = False)
  
  test_report, human_pred = ajf.test_model(model, dev_iter,
                                          input_names, target_name,
                                          saved_model_path,
                                          LABEL.vocab, temp_path,
                                          #operations = {"confusion","class_metrics", "average_schemes"},
                                          operations = {"average_schemes"},
                                          gold=GOLD, verbosity=True)
  return train_report, test_report, human_pred

def experiment_pipeline(grid_search, key, hopt, model_kind,
                       verbosity = True):
  runs_report = pd.DataFrame()
  watch = ajutil.Stopwatch()
  if verbosity: print("\n".join([f'Starting @ {watch.current_time()} with',
                   f'epochs = {hopt["EPOCHS"]};  avg = {METRIC_AVERAGE}'
                                ])
                     )
  for i, parm in enumerate(grid_search):
    display(Markdown(f'<hr><b>Run {i+1}/{len(grid_search)}</b>: {key}={parm} from {grid_search}'))
    
    # These 2 lines are the very important ones for grid_search,
    # everything else is for reporting
    hopt[key] = parm
    train_report, test_report, human_pred = deploy_experiment(*model_creation(model_kind,True))

    assert GOLD, "No gold data, not use this cycle! call your function directly"
    run_time = watch.stop()
    if verbosity: print(f'end_run {i+1}/{len(grid_search)}: '
                        f'⏲{run_time}, @ {watch.current_time()}'
                       )      
    
    search_report = pd.Series([parm, run_time], index=[key,"⏲"])
    train_test_report = train_report.append(test_report)
    runs_report = runs_report.append(search_report.append(train_test_report),
                                     ignore_index=True)

  total_report = pd.Series([grid_search, watch.totaltime],
                           index=[key,"⏲"], name="total")
  runs_report = runs_report.append(total_report, ignore_index=True)
  runs_report.to_csv(temp_path+"runs_report", index=False)
  
  if verbosity: display(Markdown("<hr><H2>FINAL REPORT:"),
                        HTML(runs_report.to_html(index=False))
                       )
    
  return train_report, test_report, human_pred, runs_report

In [20]:
grid_search, key = list(range(3)), "RUN"

#grid_search, key = list(range(10,10*5+1,5)), "EPOCHS"
#grid_search, key = np.arange(0.1, 0.9, 0.1), "DROPOUT"
#grid_search, key = list(range(16,16*4+1, 16)), "BATCH_SIZE"

model_kind = "bert" #cnn gru bert bert_gru

# full_reports contains: train_report, test_report, human_pred, runs_report
full_reports = experiment_pipeline(grid_search, key, hopt, model_kind)


Starting @ 04:54:38 with
epochs = 1;  avg = weighted


<hr><b>Run 1/3</b>: RUN=0 from [0, 1, 2]

The model has 216,089 trainable parameters


/001/usuarios/ajason08/anaconda3/envs/nlp_py369/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**> Average schemes comparison**:

...

metric \ scheme,micro,macro,weighted
precision,0.318,0.002558,0.104651
recall,0.318,0.005058,0.318000
fscore,0.318,0.002700,0.154920


end_run 2/3: ⏲3m 36s, @ 05:01:51


<hr><b>Run 3/3</b>: RUN=2 from [0, 1, 2]

The model has 216,089 trainable parameters


KeyboardInterrupt: 

In [ ]:
hopt["BERT_FROZEN"]